In [11]:
import pandas as pd

# users = pd.read_csv('data/users_export.csv')
ratings = pd.read_csv('data/ratings_export.csv')
movies = pd.read_csv('data/movie_data.csv', on_bad_lines='skip')
my_ratings = pd.read_csv('data/processed/ratings_tmdb_cleaned.csv')

# Merge datasets to create a comprehensive dataset
user_ratings = pd.merge(ratings, movies, left_on='movie_id', right_on='movie_id')

In [12]:
user_ratings.head()

,_id_x,movie_id,rating_val,user_id,_id_y,genres,image_url,imdb_id,imdb_link,movie_title,...,popularity,production_countries,release_date,runtime,spoken_languages,tmdb_id,tmdb_link,vote_average,vote_count,year_released
0,5fc57c5d6758f6963451a07f,feast-2014,7,deathproof,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
1,5fc57ca06758f69634538bad,feast-2014,10,ingridgoeswest,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
2,5fc57cbd6758f696345475a3,feast-2014,8,dirkh,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
3,5fc57ce06758f6963455400b,feast-2014,10,childrenofmen,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
4,5fc57cf36758f69634558b0e,feast-2014,8,suspirliam,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0


In [13]:
len(user_ratings)

11079666

In [14]:
len(my_ratings)

186

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

# Assuming 'tmdb_id' in movies DataFrame and 'id' in ratings DataFrame are the columns to be used
# The 'left_on' and 'right_on' parameters in merge function need to be updated accordingly

# Updated merge to use 'tmdb_id' from movies DataFrame
merged = my_ratings.merge(movies[['tmdb_id', 'movie_title', 'year_released']], 
                          left_on='id',  # Update this based on the actual column in my_ratings that matches tmdb_id in movies
                          right_on='tmdb_id', 
                          how='inner')

# Assuming that 'id' in my_ratings corresponds to 'id' in the ratings DataFrame (update if necessary)
# Here, we directly use 'tmdb_id' instead of 'movie_id' and assign 'user_id' = 0
my_ratings_updated = merged[['tmdb_id', 'Rating']].copy()  # Use .copy() to explicitly create a copy
my_ratings_updated['user_id'] = 0  # Assign 'user_id' = 0 directly

# Aggregate ratings for each movie (in case of duplicates), using 'tmdb_id'
my_ratings_updated = my_ratings_updated.groupby('tmdb_id').agg({'Rating': 'mean'}).reset_index()

# Output the updated ratings DataFrame
for i in range(len(my_ratings_updated)):
    print(my_ratings_updated.iloc[i]['Name'], my_ratings_updated.iloc[i]['Rating'])

# Output the number of unique movies rated
len(my_ratings_updated)


11.0 4.5
12.0 4.5
85.0 3.5
89.0 3.5
105.0 4.0
137.0 4.0
165.0 0.0
196.0 0.0
218.0 4.0
280.0 3.5
329.0 3.5
496.0 0.0
508.0 4.0
585.0 4.0
601.0 0.0
603.0 0.0
607.0 4.0
620.0 0.0
630.0 4.0
640.0 4.0
671.0 0.0
672.0 0.0
673.0 0.0
674.0 0.0
675.0 0.0
686.0 4.0
772.0 0.0
808.0 0.0
809.0 0.0
854.0 0.0
861.0 0.0
862.0 0.0
863.0 0.0
920.0 5.0
1669.0 0.0
1724.0 0.0
1726.0 4.5
1771.0 0.0
1885.0 0.0
1891.0 0.0
1892.0 4.0
1893.0 0.0
1894.0 0.0
1895.0 0.0
2062.0 4.0
2109.0 0.0
2493.0 0.0
4011.0 0.0
5255.0 0.0
8835.0 3.5
9340.0 0.0
9479.0 0.0
9502.0 4.0
9806.0 0.0
9919.0 4.5
10138.0 0.0
10191.0 0.0
10193.0 4.5
10195.0 0.0
10315.0 0.0
10681.0 4.5
10719.0 0.0
10947.0 0.0
11036.0 4.5
12445.0 4.0
14160.0 0.0
18240.0 2.5
18785.0 3.0
19913.0 4.5
19995.0 0.0
20352.0 0.0
24428.0 0.0
27205.0 4.5
37165.0 3.0
37724.0 0.0
37735.0 0.0
37799.0 0.0
38757.0 0.0
41630.0 4.0
49013.0 4.0
49521.0 0.0
49530.0 0.0
50544.0 3.5
50646.0 3.5
59967.0 3.0
62211.0 0.0
64328.0 0.0
68721.0 0.0
72105.0 3.5
73723.0 0.0
75656.0 4.0
7

176

In [16]:

# Add your ratings to the user ratings DataFrame
combined_ratings = pd.concat([user_ratings, my_ratings_updated.rename(columns={'Rating': 'rating_val'})])

# Map movie IDs and user IDs to indices for creating a sparse matrix
movie_id_to_idx = {movie_id: i for i, movie_id in enumerate(combined_ratings['movie_id'].unique())}
user_id_to_idx = {user_id: i for i, user_id in enumerate(combined_ratings['user_id'].unique())}

# Create a sparse matrix of ratings
rows = combined_ratings['user_id'].map(user_id_to_idx)
cols = combined_ratings['movie_id'].map(movie_id_to_idx)
data = combined_ratings['rating_val']
ratings_matrix = csr_matrix((data, (rows, cols)), shape=(len(user_id_to_idx), len(movie_id_to_idx)))

# Compute cosine similarity between users
user_similarity = cosine_similarity(ratings_matrix)
user_similarity

array([[1.        , 0.49803018, 0.34356284, ..., 0.01531783, 0.02144497,
        0.        ],
       [0.49803018, 1.        , 0.36897997, ..., 0.        , 0.        ,
        0.        ],
       [0.34356284, 0.36897997, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.01531783, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.02144497, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [17]:
# Since you are user_id 0, find the most similar users to you
top_100_similar_users_indices = np.argsort(-user_similarity[0])[1:101]  # Get indices of top 100 similar users

my_rated_movies = set(my_ratings_updated['movie_id'])

# Initialize a dictionary to hold recommended movies and the users who recommended them
recommended_movies_details = {}

for user_index in top_100_similar_users_indices:
    user_id = list(user_id_to_idx.keys())[list(user_id_to_idx.values()).index(user_index)]
    
    # # Filter movies rated 5/5 by the user
    # high_rated_movies_by_user = combined_ratings[(combined_ratings['user_id'] == user_id) & (combined_ratings['rating_val'] == 5)]
    high_rated_movies_by_user = combined_ratings[(combined_ratings['user_id'] == user_id)]
    
    for _, row in high_rated_movies_by_user.iterrows():
        movie_id = row['movie_id']
        if movie_id not in my_rated_movies:
            if movie_id not in recommended_movies_details:
                recommended_movies_details[movie_id] = {'users': [user_id], 'ratings': [row['rating_val']]}
            else:
                recommended_movies_details[movie_id]['users'].append(user_id)
                recommended_movies_details[movie_id]['ratings'].append(row['rating_val'])

# Limit to top 5 based on the number of users recommending the movie
recommended_movies_ids = sorted(recommended_movies_details, key=lambda x: len(recommended_movies_details[x]['users']), reverse=True)[:100]

# Fetch movie titles and stats
recommended_titles_and_stats = []
for movie_id in recommended_movies_ids:
    movie_title = movies[movies['movie_id'] == movie_id]['movie_title'].iloc[0]
    avg_rating = np.mean(recommended_movies_details[movie_id]['ratings'])
    num_users = len(recommended_movies_details[movie_id]['users'])
    recommended_titles_and_stats.append({
        'title': movie_title,
        'average_rating': avg_rating,
        'recommended_by_users_count': num_users,
        'recommended_by_user_ids': recommended_movies_details[movie_id]['users']
    })

# Display recommended movies along with stats
for movie in recommended_titles_and_stats:
    print(f"Title: {movie['title']}, Avg Rating: {movie['average_rating']:.2f}, Recommended by {movie['recommended_by_users_count']} Users, User IDs: {movie['recommended_by_user_ids']}")

KeyError: 'movie_id'

In [ ]:
# output to file

with open('data/recommended_movies.txt', 'w') as f:
    for movie in recommended_titles_and_stats:
        f.write(f"Title: {movie['title']}, Avg Rating: {movie['average_rating']:.2f}, Recommended by {movie['recommended_by_users_count']} Users, User IDs: {movie['recommended_by_user_ids']}\n")